## Classifier deception
Code has been taken from [here](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
image_datasets = {x: datasets.ImageFolder("./" + x,
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=4,
                                             shuffle=True, num_workers=4),
               'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=1,
                                             shuffle=False, num_workers=4)
              }
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model

In [5]:
def get_error_mask(model):
    was_training = model.training
    model.eval()
    
    error_mask = []
    
    with torch.no_grad():
        for i, (input, label) in enumerate(dataloaders['test']):
            input = input.to(device)
            label = label.to(device)

            outputs = model(input)
            _, pred = torch.max(outputs, 1)
            
            true = class_names[label]
            pred = class_names[pred]

            error_mask.append(true != pred)

        model.train(mode=was_training)
        return error_mask

In [6]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Only classifier's parameters are being optimized
optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [7]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)

Epoch 1/10
----------
train Loss: 0.8409 Acc: 0.5256
test Loss: 1.0588 Acc: 0.5088

Epoch 2/10
----------
train Loss: 0.5354 Acc: 0.7179
test Loss: 0.4656 Acc: 0.7018

Epoch 3/10
----------
train Loss: 0.5086 Acc: 0.7650
test Loss: 0.4728 Acc: 0.7544

Epoch 4/10
----------
train Loss: 0.4679 Acc: 0.8077
test Loss: 0.3030 Acc: 0.8772

Epoch 5/10
----------
train Loss: 0.3709 Acc: 0.8248
test Loss: 0.2877 Acc: 0.8947

Epoch 6/10
----------
train Loss: 0.3762 Acc: 0.8248
test Loss: 0.3067 Acc: 0.8421

Epoch 7/10
----------
train Loss: 0.3871 Acc: 0.8120
test Loss: 0.3130 Acc: 0.8070

Epoch 8/10
----------
train Loss: 0.3708 Acc: 0.8419
test Loss: 0.2812 Acc: 0.8421

Epoch 9/10
----------
train Loss: 0.3196 Acc: 0.8718
test Loss: 0.2887 Acc: 0.8772

Epoch 10/10
----------
train Loss: 0.3191 Acc: 0.8803
test Loss: 0.2742 Acc: 0.8772

Training complete in 9m 5s


In [8]:
mask = get_error_mask(model_ft)

In [9]:
errors = np.array(image_datasets['test'].imgs)[mask][:,0]
errors = [e[e.rindex('\\')+1:] for e in errors]
for model in ['cg', 'nst', 'cnnmrf', 'ours']:
    error = len([e for e in errors if model in e]) / len(image_datasets['test'].imgs)
    print(f'{model}: {error*100:.2f}%')
    
error = len([e for e in errors if model not in e]) / len(image_datasets['test'].imgs)
print(f'other: {error*100:.2f}%')

cg: 0.00%
nst: 0.00%
cnnmrf: 0.00%
ours: 3.51%
other: 8.77%
